In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_0.npz')
test_data = np.load('./micro-wavllm-large/test_0.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_1.npz')
test_data = np.load('./micro-wavllm-large/test_1.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_2.npz')
test_data = np.load('./micro-wavllm-large/test_2.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_3.npz')
test_data = np.load('./micro-wavllm-large/test_3.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_4.npz')
test_data = np.load('./micro-wavllm-large/test_4.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_5.npz')
test_data = np.load('./micro-wavllm-large/test_5.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_6.npz')
test_data = np.load('./micro-wavllm-large/test_6.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_7.npz')
test_data = np.load('./micro-wavllm-large/test_7.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_8.npz')
test_data = np.load('./micro-wavllm-large/test_8.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_9.npz')
test_data = np.load('./micro-wavllm-large/test_9.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_10.npz')
test_data = np.load('./micro-wavllm-large/test_10.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_11.npz')
test_data = np.load('./micro-wavllm-large/test_11.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_12.npz')
test_data = np.load('./micro-wavllm-large/test_12.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_13.npz')
test_data = np.load('./micro-wavllm-large/test_13.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_14.npz')
test_data = np.load('./micro-wavllm-large/test_14.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_15.npz')
test_data = np.load('./micro-wavllm-large/test_15.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_16.npz')
test_data = np.load('./micro-wavllm-large/test_16.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_17.npz')
test_data = np.load('./micro-wavllm-large/test_17.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_18.npz')
test_data = np.load('./micro-wavllm-large/test_18.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_19.npz')
test_data = np.load('./micro-wavllm-large/test_19.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_20.npz')
test_data = np.load('./micro-wavllm-large/test_20.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_21.npz')
test_data = np.load('./micro-wavllm-large/test_21.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_22.npz')
test_data = np.load('./micro-wavllm-large/test_22.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_23.npz')
test_data = np.load('./micro-wavllm-large/test_23.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./micro-wavllm-large/train_24.npz')
test_data = np.load('./micro-wavllm-large/test_24.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# CNN Predictions and Classification Report
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
print("CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")
